# Scrape wikipedia page.

Use the Notebook to build the code to scrape the following Wikipedia page,
- https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

### Imports

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

### Scrape table using pandas

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url).content
codes=pd.read_html(results)[0]

### Clean Not assigned boroughs.

In [4]:
p_codes=codes[codes['Borough']!='Not assigned']

### Check if Not assigned neighbourhood exist

In [5]:
test=p_codes[p_codes['Neighbourhood']=='Not assigned']
print(test.count())

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64


In [6]:
p_codes.shape

(210, 3)

In [10]:
#!pip install geocoder
#import geocoder # import geocoder

In [27]:
# loop until you get the coordinates
#column_names=['Postcode','Lattitude','Longitude']
#latlong=pd.DataFrame(columns=column_names);
#for postal_code in p_codes['Postcode']:
#    lat_lng_coords = None
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#      lat_lng_coords = g.latlng
#    print(lat_lng_coords)
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
#    latlong.append({'Postcode':postal_code,'Lattitude':latitude,'Longitude':longitude},ignore_index=True)

# Get lattitude and longitude coodinates.

In [14]:
csv_path='http://cocl.us/Geospatial_data'
lat_long=pd.read_csv(csv_path)
lat_long.rename(columns={'Postal Code':'Postcode'},inplace=True)
lat_long.head()

### Merge lattitude and longitude to neighbourhood dataframe.

In [28]:
merged=p_codes.join(lat_long.set_index('Postcode'),on='Postcode')
merged.reset_index()
merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M6A,North York,Lawrence Heights,43.718518,-79.464763
6,M6A,North York,Lawrence Manor,43.718518,-79.464763
